In [580]:
import json

In [581]:
f = open('model.json')

model = json.load(f)

f.close()

model

{'meta': {'name': 'Cloud Description',
  'description': 'Описание признаков облаков',
  'schemaVersion': '1'},
 'schema': [{'param': 'ground_temperature',
   'title': 'Температура у поверхности',
   'description': 'Текущая температура у поверхности (или над водой)',
   'unit': 'C°',
   'spec': {'type': 'float', 'min': '-200.0', 'max': '100.0'},
   'require': True},
  {'param': 'height',
   'title': 'Высота облаков',
   'description': 'Оценочная высота облаков в метрах',
   'unit': 'm',
   'spec': {'type': 'int', 'min': '0', 'max': '100000'},
   'require': False},
  {'param': 'thickness',
   'title': 'Толщина облаков',
   'description': 'Оценочная толщина облаков в метрах',
   'unit': 'm',
   'spec': {'type': 'int', 'min': '1', 'max': '1000'},
   'require': False},
  {'param': 'color',
   'title': 'Цвет облаков',
   'description': 'Оценочный цвет облаков',
   'spec': {'type': 'enum',
    'data': [{'title': 'Белый',
      'description': 'Чистый и белый цвет',
      'image': 'sample.jpg',

In [582]:
f = open('data.json')

data = json.load(f)

f.close()

data

{'meta': {'name': 'Cloud Classificator',
  'description': 'Классификатор облаков на основании признаков',
  'schemaVersion': '1'},
 'stack': [{'id': 'type',
   'title': 'Тип облаков',
   'description': 'Тип облака определяется условием образования',
   'limit': 1},
  {'id': 'morphology',
   'title': 'Морфологическая классификация',
   'description': 'На основании структуры и внешнего вида облака'},
  {'id': 'attention',
   'title': 'Предупреждение о опасностях',
   'description': 'Возможные опасные погодные явления, сопутствующие данному виду облаков'}],
 'graph': [{'type': 'condition',
   'comment': 'ТИП: Конвективные облака',
   'statements': [{'type': 'statement',
     'param': 'ground_temperature',
     'predicate': '>',
     'value': '20',
     'require': False,
     'score': 1}],
   'graph': [{'type': 'report',
     'stack': 'type',
     'report': {'title': 'Конвективные облака',
      'description': 'Формируются за счёт неравномерного нагрева снизу и восходящего тока более тёплы

In [583]:
obj = {}

for param in model['schema']:
    obj[param['param']] = None

obj

{'ground_temperature': None,
 'height': None,
 'thickness': None,
 'color': None,
 'speed': None,
 'precipitation': None,
 'lightning': None,
 'visual_morph': None}

In [584]:
obj['height'] = 50
# obj['ground_temperature'] = 50
obj['color'] = 'gray'
# obj['visual_morph'] = 'cirrocumulus'
obj

{'ground_temperature': None,
 'height': 50,
 'thickness': None,
 'color': 'gray',
 'speed': None,
 'precipitation': None,
 'lightning': None,
 'visual_morph': None}

In [585]:
stack = {}

def stack_flush():
    for s in data['stack']:
        stack[s['id']] = []

def stack_result(id, result, score):
    result['score'] = score
    stack[id].append(result)

In [586]:
class Node():
    def __init__(self, graph, depth = 0):
        self.type = graph['type']
        self.graph = graph
        self.nodes = []
        self.depth = depth
        self.score = 0
        
        try:
            self.stack = graph['stack']
        except:
            self.stack = None
        
        try:
            self.comment = graph['comment']
        except:
            self.comment = None

        try:
            for g in graph['graph']:
                self.nodes.append(Node(g, depth + 1))
        except:
            pass


        match self.type:
            case 'condition':
                self.statements = graph['statements']
            case 'report':
                self.report = graph['report']

    def debug_trace(self, trace = []):
        trace.append(self)
        for node in self.nodes:
            node.debug_trace(trace)

        return trace
    
    def debug_info(self):
        return {
            'node_count': len(self.nodes), 
            'depth': self.depth,
            'type': self.type
        }
    
    def debug_tree(self):
        for n in self.debug_trace():
            info = n.debug_info()
            print('     ' * n.depth + str(info))

    def execute_condition(self):
        true_count = 0
        global_score = 0

        for statement in self.statements:
            try:
                st_score = statement['score']
            except:
                st_score = 0


            if statement['param'] in obj:
                if obj[statement['param']] is not None:
                    values = []
                    if type(statement['value']) is not list:
                        values.append(statement['value'])
                    else:
                        values = statement['value']
                    
                    for dst in values:
                        ost = obj[statement['param']]

                        match statement['predicate']:
                            case '>':
                                if float(ost) > float(dst):
                                    true_count += 1
                                    global_score += st_score
                            case '<':
                                if float(ost) < float(dst):
                                    true_count += 1
                                    global_score += st_score
                            case '=':
                                if ost == dst:
                                    true_count += 1
                                    global_score += st_score
                            case '>=':
                                if float(ost) >= float(dst):
                                    true_count += 1
                                    global_score += st_score
                            case '<=':
                                if float(ost) <= float(dst):
                                    true_count += 1
                                    global_score += st_score
                            case '!=':
                                if float(ost) != float(dst):
                                    true_count += 1
                                    global_score += st_score
                            case _:
                                pass
                else:
                    if not statement['require']:
                        true_count += 1
        
        if true_count > 0:
            self.execute_nodes(global_score + self.score)

    def execute(self, score = 0):
        self.score = score

        match self.type:
            case 'report':
                if 'requireScore' in self.report:
                    if self.report['requireScore'] <= self.score:
                        stack_result(self.stack, self.report, self.score)
                        self.execute_nodes(self.score)
                else:
                    stack_result(self.stack, self.report, self.score)
                    self.execute_nodes(self.score)
            case 'condition':
                self.execute_condition()

            # case _:
            #     self.execute_nodes(self.score)
    
    def execute_nodes(self, score):
        for n in self.nodes:
            n.execute(score)


In [587]:
n = Node(data['graph'][0])
n.debug_tree()

{'node_count': 3, 'depth': 0, 'type': 'condition'}
     {'node_count': 0, 'depth': 1, 'type': 'report'}
     {'node_count': 1, 'depth': 1, 'type': 'condition'}
          {'node_count': 0, 'depth': 2, 'type': 'report'}
     {'node_count': 1, 'depth': 1, 'type': 'condition'}
          {'node_count': 0, 'depth': 2, 'type': 'report'}


In [588]:
stack_flush()
n.execute()

print(json.dumps(stack, indent=4, ensure_ascii=False))

{
    "type": [
        {
            "title": "Конвективные облака",
            "description": "Формируются за счёт неравномерного нагрева снизу и восходящего тока более тёплых воздушных масс.",
            "score": 0
        }
    ],
    "morphology": [
        {
            "title": "Кучево-дождевые облака",
            "description": "Хорошо развитые по вертикали конвективные облака в виде плотной массы с тёмно-серым или чёрным основанием, под которым видны полосы падения осадков.",
            "requireScore": 2,
            "score": 10
        }
    ],
    "attention": []
}
